In [137]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [10]:
odds = pd.read_excel('C:\\Users\\Daniel\\Documents\\Flatiron\\Capstone\\sports_predictions\\nba_odds.xlsx')
odds.head()

,Date,Season,Home,Away,Home Win By (Vegas),Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1228,2020,Atlanta,Detroit,10.5,NaN,NaN,NaN,NaN,NaN,NaN
1,102,2020,Atlanta,Cleveland,5.5,NaN,NaN,NaN,NaN,NaN,NaN
2,104,2020,Atlanta,NewYork,7,NaN,NaN,NaN,NaN,NaN,NaN
3,106,2020,Atlanta,Charlotte,6,NaN,NaN,NaN,NaN,NaN,NaN
4,111,2020,Atlanta,Philadelphia,6,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
odds.drop(columns=odds.columns[5:], inplace=True)

In [12]:
odds.head()

,Date,Season,Home,Away,Home Win By (Vegas)
0,1228,2020,Atlanta,Detroit,10.5
1,102,2020,Atlanta,Cleveland,5.5
2,104,2020,Atlanta,NewYork,7
3,106,2020,Atlanta,Charlotte,6
4,111,2020,Atlanta,Philadelphia,6


In [13]:
#split date column into month and day
days = []
months = []
for i in range(len(odds)):
    day = str(odds['Date'][i])[-2:]
    month = str(odds['Date'][i])[:-2]
    days.append(day)
    months.append(month)

odds['day'] = days
odds['month'] = months

odds.head()

,Date,Season,Home,Away,Home Win By (Vegas),day,month
0,1228,2020,Atlanta,Detroit,10.5,28,12
1,102,2020,Atlanta,Cleveland,5.5,02,1
2,104,2020,Atlanta,NewYork,7,04,1
3,106,2020,Atlanta,Charlotte,6,06,1
4,111,2020,Atlanta,Philadelphia,6,11,1


In [14]:
#derive the year based on day/month/season
years = []
for i in range(len(odds)):
    if odds['Date'][i] < 1012:
        year = odds['Season'][i]+1
    else:
        year = odds['Season'][i]
    
    years.append(year)

odds['year'] = years

odds.head()

,Date,Season,Home,Away,Home Win By (Vegas),day,month,year
0,1228,2020,Atlanta,Detroit,10.5,28,12,2020
1,102,2020,Atlanta,Cleveland,5.5,02,1,2021
2,104,2020,Atlanta,NewYork,7,04,1,2021
3,106,2020,Atlanta,Charlotte,6,06,1,2021
4,111,2020,Atlanta,Philadelphia,6,11,1,2021


In [15]:
#create date column
dates = []
for i in range(len(odds)):
    date = pd.to_datetime('{}-{}-{}'.format(odds['month'][i], odds['day'][i], odds['year'][i]), format='%m-%d-%Y')
    dates.append(date)

odds['Date'] = dates
odds.head()

,Date,Season,Home,Away,Home Win By (Vegas),day,month,year
0,2020-12-28,2020,Atlanta,Detroit,10.5,28,12,2020
1,2021-01-02,2020,Atlanta,Cleveland,5.5,02,1,2021
2,2021-01-04,2020,Atlanta,NewYork,7,04,1,2021
3,2021-01-06,2020,Atlanta,Charlotte,6,06,1,2021
4,2021-01-11,2020,Atlanta,Philadelphia,6,11,1,2021


In [16]:
odds.drop(columns=['day','month','year'], inplace=True)

In [19]:
odds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11656 entries, 0 to 11655
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 11656 non-null  datetime64[ns]
 1   Season               11656 non-null  int64         
 2   Home                 11656 non-null  object        
 3   Away                 11656 non-null  object        
 4   Home Win By (Vegas)  11656 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 455.4+ KB


In [22]:
#collect all the url links for games played on everyday from 2011-2021
links = []
dates = []
for date in odds['Date'].unique():
    date = pd.to_datetime(date)
    html_page = requests.get('https://www.basketball-reference.com/boxscores/?month={}&day={}&year={}'
                             .format(date.month, date.day, date.year))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
    soups = soup.find_all('p', {'class': 'links'})
    for i in range(len(soups)):
        entry = soups[i]
        link = entry.find('a')['href']
        links.append(link)
        dates.append('{}-{}-{}'.format(date.year,date.month,date.day))
len(links)

11656

In [59]:
#because of inconsistent stat keeping between sites for advanced metrics, we are going to use orb, drb, tov, stl, and blk,
#instead of their respective %s
#we are also no longer doing ortg and drtg as they directly correlate with points for and points against
#we are adding ftm and fta to get a ft%, as well as 3PAr 
df = pd.DataFrame(columns=['Date','Home Team','Home Points For','Home Points Against','Home eFG%','Home FTr', 'Home FTM', 
                           'Home FTA','Home 3PAr','Home ORB', 'Home DRB','Home AST%','Home STL','Home BLK', 'Home TOV',
                           'Away Team','Away Points For','Away Points Against','Away eFG%','Away FTr', 'Away FTM', 'Away FTA',
                           'Away 3PAr','Away ORB', 'Away DRB','Away AST%','Away STL', 'Away BLK', 'Away TOV'],
                  index=range(len(dates)))

In [54]:
html_page = requests.get('https://www.basketball-reference.com/boxscores/202111180CLE.html')
soup = BeautifulSoup(html_page.content, 'html.parser')

soup('table')[int(len(soup('table'))/2)]('tr')[-1]('td')[8].getText()

'16'

In [56]:
soup('table')[0]('tr')[-1]('td')[8].getText()

'8'

In [60]:
#for loop to scrape all the data
for i, link in enumerate(links):
    html_page = requests.get('https://www.basketball-reference.com{}'.format(link))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    strong = soup('strong')
    table = soup('table')
    df['Date'][i] = dates[i]
    
    df['Home Team'][i] = strong[2].getText()[1:-1]
    df['Home Points For'][i] = table[int(len(table)/2)]('tr')[-1]('td')[-2].getText()
    df['Home Points Against'][i] = table[0]('tr')[-1]('td')[-2].getText()
    df['Home eFG%'][i] = table[-1]('tr')[-1]('td')[2].getText()
    df['Home FTr'][i] = table[-1]('tr')[-1]('td')[4].getText()
    df['Home FTM'][i] = table[int(len(table)/2)]('tr')[-1]('td')[7].getText()
    df['Home FTA'][i] = table[int(len(table)/2)]('tr')[-1]('td')[8].getText()
    df['Home 3PAr'][i] = table[-1]('tr')[-1]('td')[3].getText()
    df['Home ORB'][i] = table[int(len(table)/2)]('tr')[-1]('td')[10].getText()
    df['Home DRB'][i] = table[int(len(table)/2)]('tr')[-1]('td')[11].getText()
    df['Home AST%'][i] = table[-1]('tr')[-1]('td')[8].getText()
    df['Home STL'][i] = table[int(len(table)/2)]('tr')[-1]('td')[14].getText()
    df['Home BLK'][i] = table[int(len(table)/2)]('tr')[-1]('td')[15].getText()
    df['Home TOV'][i] = table[int(len(table)/2)]('tr')[-1]('td')[16].getText()
    
    df['Away Team'][i] = strong[1].getText()[1:-1]
    df['Away Points For'][i] = table[0]('tr')[-1]('td')[-2].getText()
    df['Away Points Against'][i] = table[int(len(table)/2)]('tr')[-1]('td')[-2].getText()
    df['Away eFG%'][i] = table[int(len(table)/2)-1]('tr')[-1]('td')[2].getText()
    df['Away FTr'][i] = table[int(len(table)/2)-1]('tr')[-1]('td')[4].getText()
    df['Away FTM'][i] = table[0]('tr')[-1]('td')[7].getText()
    df['Away FTA'][i] = table[0]('tr')[-1]('td')[8].getText()
    df['Away 3PAr'][i] = table[int(len(table)/2)-1]('tr')[-1]('td')[3].getText()
    df['Away ORB'][i] = table[0]('tr')[-1]('td')[10].getText()
    df['Away DRB'][i] = table[0]('tr')[-1]('td')[11].getText()
    df['Away AST%'][i] = table[int(len(table)/2)-1]('tr')[-1]('td')[8].getText()
    df['Away STL'][i] = table[0]('tr')[-1]('td')[14].getText()
    df['Away BLK'][i] = table[0]('tr')[-1]('td')[15].getText()
    df['Away TOV'][i] = table[0]('tr')[-1]('td')[16].getText()
    
df.head()

,Date,Home Team,Home Points For,Home Points Against,Home eFG%,Home FTr,Home FTM,Home FTA,Home 3PAr,Home ORB,...,Away FTr,Away FTM,Away FTA,Away 3PAr,Away ORB,Away DRB,Away AST%,Away STL,Away BLK,Away TOV
0,2020-12-28,Atlanta Hawks,128,120,.610,.390,28,32,.512,8,...,.245,21,24,.418,17,31,55.8,7,4,8
1,2020-12-28,Brooklyn Nets,111,116,.480,.170,15,17,.420,13,...,.296,22,29,.296,16,40,47.6,10,4,19
2,2020-12-28,Denver Nuggets,124,111,.609,.368,18,32,.356,11,...,.267,17,23,.453,5,33,64.1,8,3,15
3,2020-12-28,Los Angeles Lakers,107,115,.529,.207,15,18,.310,6,...,.163,13,15,.500,7,38,62.8,9,6,9
4,2020-12-28,Oklahoma City Thunder,109,110,.552,.221,14,19,.442,7,...,.163,11,14,.419,8,35,55.8,9,6,16


In [63]:
#standardizing team names between odds and stats data so they can be combined later
for i in range(len(df)):
    for place in ['Home Team', 'Away Team']:
        if df[place][i] == 'Los Angeles Lakers':
            df[place][i] = 'Lakers'
        elif df[place][i] == 'Los Angeles Clippers':
            df[place][i] = 'Clippers'
        else:    
            df[place][i] = df[place][i].replace(' {}'.format(df[place][i].split()[-1]),'')
df.head()

,Date,Home Team,Home Points For,Home Points Against,Home eFG%,Home FTr,Home FTM,Home FTA,Home 3PAr,Home ORB,...,Away FTr,Away FTM,Away FTA,Away 3PAr,Away ORB,Away DRB,Away AST%,Away STL,Away BLK,Away TOV
0,2020-12-28,Atlanta,128,120,.610,.390,28,32,.512,8,...,.245,21,24,.418,17,31,55.8,7,4,8
1,2020-12-28,Brooklyn,111,116,.480,.170,15,17,.420,13,...,.296,22,29,.296,16,40,47.6,10,4,19
2,2020-12-28,Denver,124,111,.609,.368,18,32,.356,11,...,.267,17,23,.453,5,33,64.1,8,3,15
3,2020-12-28,Lakers,107,115,.529,.207,15,18,.310,6,...,.163,13,15,.500,7,38,62.8,9,6,9
4,2020-12-28,Oklahoma City,109,110,.552,.221,14,19,.442,7,...,.163,11,14,.419,8,35,55.8,9,6,16


In [64]:
#standardize odds data
for i in range(len(odds)):
    for place in ['Home', 'Away']:
        if odds[place][i] == 'GoldenState':
            odds[place][i] = 'Golden State'
        elif odds[place][i] == 'LAClippers':
            odds[place][i] = 'Clippers'
        elif odds[place][i] == 'LALakers':
            odds[place][i] = 'Lakers'
        elif odds[place][i] == 'NewOrleans':
            odds[place][i] = 'New Orleans'
        elif odds[place][i] == 'NewYork':
            odds[place][i] = 'New York'
        elif odds[place][i] == 'OklahomaCity':
            odds[place][i] = 'Oklahoma City'
        elif odds[place][i] == 'SanAntonio':
            odds[place][i] = 'San Antonio'
        elif odds[place][i] == 'NewJersey':
            odds[place][i] = 'NewJersey'
odds.head()

<ipython-input-64-ff79eb82b092>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds[place][i] = 'New York'
<ipython-input-64-ff79eb82b092>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds[place][i] = 'Clippers'
<ipython-input-64-ff79eb82b092>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds[place][i] = 'Lakers'
<ipython-input-64-ff79eb82b092>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,Date,Season,Home,Away,Home Win By (Vegas)
0,2020-12-28,2020,Atlanta,Detroit,10.5
1,2021-01-02,2020,Atlanta,Cleveland,5.5
2,2021-01-04,2020,Atlanta,New York,7
3,2021-01-06,2020,Atlanta,Charlotte,6
4,2021-01-11,2020,Atlanta,Philadelphia,6


In [67]:
odds['Home'].unique().sort() == df['Home Team'].unique().sort()

True

In [68]:
odds['Away'].unique().sort() == df['Away Team'].unique().sort()

True

In [70]:
odds.rename(columns={'Home': 'Home Team', 'Away': 'Away Team'}, inplace=True)

In [84]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [85]:
#create 'gamedID' column and set as index to easily identify games
for table in [odds, df]:
    ids = []
    for i in range(len(table)):
        game_id = '{} {} @ {}'.format(table['Date'][i], table['Away Team'][i], table['Home Team'][i])
        ids.append(game_id)

    table['GameID'] = ids
    table.set_index('GameID', drop=True, inplace=True)

In [86]:
df.head()

,Date,Home Team,Home Points For,Home Points Against,Home eFG%,Home FTr,Home FTM,Home FTA,Home 3PAr,Home ORB,...,Away FTr,Away FTM,Away FTA,Away 3PAr,Away ORB,Away DRB,Away AST%,Away STL,Away BLK,Away TOV
GameID,,,,,,,,,,,,,,,,,,,,,
2020-12-28 00:00:00 Detroit @ Atlanta,2020-12-28,Atlanta,128,120,.610,.390,28,32,.512,8,...,.245,21,24,.418,17,31,55.8,7,4,8
2020-12-28 00:00:00 Memphis @ Brooklyn,2020-12-28,Brooklyn,111,116,.480,.170,15,17,.420,13,...,.296,22,29,.296,16,40,47.6,10,4,19
2020-12-28 00:00:00 Houston @ Denver,2020-12-28,Denver,124,111,.609,.368,18,32,.356,11,...,.267,17,23,.453,5,33,64.1,8,3,15
2020-12-28 00:00:00 Portland Trail @ Lakers,2020-12-28,Lakers,107,115,.529,.207,15,18,.310,6,...,.163,13,15,.500,7,38,62.8,9,6,9
2020-12-28 00:00:00 Utah @ Oklahoma City,2020-12-28,Oklahoma City,109,110,.552,.221,14,19,.442,7,...,.163,11,14,.419,8,35,55.8,9,6,16


In [94]:
for column in df.columns[2:15]:
    df[column] = pd.to_numeric(df[column], downcast = 'float')

In [96]:
for column in df.columns[16:]:
    df[column] = pd.to_numeric(df[column], downcast = 'float')

In [98]:
df['Home Win By'] = df['Home Points For'] - df['Home Points Against']

In [100]:
odds.sort_index(inplace=True)
df.sort_index(inplace=True)

In [101]:
odds.head()

,Date,Season,Home Team,Away Team,Home Win By (Vegas)
GameID,,,,,
2011-12-25 00:00:00 Boston @ New York,2011-12-25,2011,New York,Boston,4.5
2011-12-25 00:00:00 Chicago @ Lakers,2011-12-25,2011,Lakers,Chicago,-5
2011-12-25 00:00:00 Clippers @ Golden State,2011-12-25,2011,Golden State,Clippers,-4.5
2011-12-25 00:00:00 Miami @ Dallas,2011-12-25,2011,Dallas,Miami,-4.5
2011-12-25 00:00:00 Orlando @ Oklahoma City,2011-12-25,2011,Oklahoma City,Orlando,7.5


In [102]:
df.head()

,Date,Home Team,Home Points For,Home Points Against,Home eFG%,Home FTr,Home FTM,Home FTA,Home 3PAr,Home ORB,...,Away FTM,Away FTA,Away 3PAr,Away ORB,Away DRB,Away AST%,Away STL,Away BLK,Away TOV,Home Win By
GameID,,,,,,,,,,,,,,,,,,,,,
2011-12-25 00:00:00 Boston @ New York,2011-12-25,New York,106.0,104.0,0.534,0.459,27.0,34.0,0.270,8.0,...,24.0,31.0,0.066,13.0,28.0,71.800003,7.0,5.0,18.0,2.0
2011-12-25 00:00:00 Chicago @ Lakers,2011-12-25,Lakers,87.0,88.0,0.500,0.263,11.0,20.0,0.211,12.0,...,9.0,14.0,0.169,16.0,25.0,58.299999,14.0,4.0,14.0,-1.0
2011-12-25 00:00:00 Clippers @ Golden State,2011-12-25,Golden State,86.0,105.0,0.421,0.293,17.0,24.0,0.256,17.0,...,23.0,38.0,0.284,13.0,30.0,59.500000,9.0,8.0,10.0,-19.0
2011-12-25 00:00:00 Miami @ Dallas,2011-12-25,Dallas,94.0,105.0,0.433,0.366,23.0,30.0,0.341,8.0,...,25.0,36.0,0.090,15.0,36.0,47.400002,9.0,6.0,22.0,-11.0
2011-12-25 00:00:00 Orlando @ Oklahoma City,2011-12-25,Oklahoma City,97.0,89.0,0.542,0.361,19.0,26.0,0.264,7.0,...,21.0,26.0,0.346,14.0,31.0,60.000000,7.0,4.0,18.0,8.0


In [115]:
#create 'Season' column to use in for loop later
for i in range(len(df)):    
    df['Season'][i] = odds['Season'][i]

<ipython-input-115-f3a132cdda59>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season'][i] = odds['Season'][i]


In [117]:
#these are the final home and away dfs, which contain each team's averages going into every game
all_home = pd.DataFrame(columns=['Home Points For','Home Points Against','Home eFG%','Home FTr', 'Home FTM', 
                                 'Home FTA','Home 3PAr','Home ORB', 'Home DRB','Home AST%','Home STL','Home BLK', 
                                 'Home TOV', 'Home GP'])

all_away = pd.DataFrame(columns=['Away Points For','Away Points Against','Away eFG%','Away FTr', 'Away FTM', 
                                 'Away FTA','Away 3PAr','Away ORB', 'Away DRB','Away AST%','Away STL', 'Away BLK', 
                                 'Away TOV', 'Away GP'])

In [141]:
#for loop to convert data from the resulting stats from each game to the avg stats going into each game
for season in range(2011,2021):
    for team in df['Home Team'].unique():
        #gather all games relevant to this team in this season
        all_df = df[(((df['Home Team']==team) & (df['Season']==season)) | 
                        ((df['Away Team']==team) & (df['Season']==season)))]
        
        #separate into home and away df's because they go in different columns at the end
        #we are going to concat axis=1
        home_all_df = all_df[all_df['Home Team']==team]
        away_all_df = all_df[all_df['Away Team']==team]
        
        #select stats relevant to home/away team
        home_df = home_all_df[home_all_df.columns[2:15]]
        away_df = away_all_df[away_all_df.columns[16:]]
        
        home_df['Home Win Rate'] = 0
        away_df['Away Win Rate'] = 0
        
        #include a win rate column
        for i in range(len(home_df)):
            if home_df['Home Points For'][i] > home_df['Home Points Against'][i]:
                home_df['Home Win Rate'][i] = 1
            else:
                home_df['Home Win Rate'][i] = 0
        
        for x in range(len(away_df)):
            if away_df['Away Points For'][x] > away_df['Away Points Against'][x]:
                away_df['Away Win Rate'][x] = 1
            else:
                away_df['Away Win Rate'][x] = 0

        #create dfs for running total throughout the season
        #also create game played column, 'GP', later to be used to take averages
        home_total = np.cumsum(home_df, axis=0)
        home_total['Home GP'] = list(range(1,len(home_df)+1))
        away_total = np.cumsum(away_df, axis=0)
        away_total['Away GP'] = list(range(1,len(away_df)+1))
        
        #create df for rolling avg stats by dividing by 'GP'
        home_avg = home_total.div(home_total['Home GP'], axis=0)
        away_avg = away_total.div(away_total['Away GP'], axis=0)
        
        #redo the 'GP' columns
        home_avg['Home GP'] = list(range(1,len(home_df)+1))
        away_avg['Away GP'] = list(range(1,len(away_df)+1))
        
        #append team's rolling avg stats for this season to aggregate df
        all_home = all_home.append(home_avg)
        all_away = all_away.append(away_avg)

<ipython-input-141-b8d5ed9ce3d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win 

C:\Users\Daniel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-141-b8d5ed9ce3d8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'][i] = 0
<ipython-input-141-b8d5ed9ce3d8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 0
<ipython-input-141-b8d5ed9ce3d

<ipython-input-141-b8d5ed9ce3d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win 

<ipython-input-141-b8d5ed9ce3d8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 0
<ipython-input-141-b8d5ed9ce3d8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 1
<ipython-input-141-b8d5ed9ce3d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:18: Set

<ipython-input-141-b8d5ed9ce3d8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 1
<ipython-input-141-b8d5ed9ce3d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away W

...

<ipython-input-141-b8d5ed9ce3d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win 

<ipython-input-141-b8d5ed9ce3d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win 

C:\Users\Daniel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-141-b8d5ed9ce3d8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'][i] = 0
<ipython-input-141-b8d5ed9ce3d8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 0
<ipython-input-141-b8d5ed9ce3d

<ipython-input-141-b8d5ed9ce3d8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'][i] = 1
<ipython-input-141-b8d5ed9ce3d8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 1
<ipython-input-141-b8d5ed9ce3d8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 0
<ipython-input-141-b8d5ed9ce3d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a

<ipython-input-141-b8d5ed9ce3d8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 0
<ipython-input-141-b8d5ed9ce3d8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df['Away Win Rate'][x] = 1
<ipython-input-141-b8d5ed9ce3d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df['Home Win Rate'] = 0
<ipython-input-141-b8d5ed9ce3d8>:18: Set

In [143]:
#combine home and away data, so that each row is a game, and 
#columns are home and away team avg stats coming into the game
all_stats = pd.concat([all_home, all_away], axis=1)

In [144]:
all_stats['Home Win By'] = df['Home Win By']

In [146]:
all_stats.tail()

,Home Points For,Home Points Against,Home eFG%,Home FTr,Home FTM,Home FTA,Home 3PAr,Home ORB,Home DRB,Home AST%,...,Away ORB,Away DRB,Away AST%,Away STL,Away BLK,Away TOV,Away GP,Home Win By,Season,Away Win Rate
2021-04-25 00:00:00 Phoenix @ Brooklyn,118.000000,111.406250,0.573563,0.271438,18.906250,23.218750,0.411531,8.750000,36.500000,63.331253,...,8.535714,32.607143,62.928567,6.892857,4.464286,12.321429,28,9.0,2020.0,0.678571
2021-04-30 00:00:00 Portland Trail @ Brooklyn,117.727273,111.909091,0.571000,0.269273,18.818182,23.090909,0.412151,8.818182,36.333333,62.927276,...,10.677419,33.677419,50.996775,6.645161,5.290323,10.774194,31,-19.0,2020.0,0.612903
2021-05-12 00:00:00 San Antonio @ Brooklyn,118.029412,112.029412,0.574324,0.271412,18.852941,23.205882,0.410441,8.647059,36.441176,63.100004,...,9.342857,34.028571,57.665712,7.228571,5.000000,10.600000,35,12.0,2020.0,0.542857
2021-05-15 00:00:00 Chicago @ Brooklyn,117.657143,111.428571,0.573686,0.268571,18.571429,22.971429,0.409886,8.628571,36.657143,63.108573,...,9.638889,35.722222,64.902778,7.138889,4.388889,14.527778,36,14.0,2020.0,0.444444
2021-05-16 00:00:00 Cleveland @ Brooklyn,117.805556,111.361111,0.576167,0.266806,18.416667,22.805556,0.408528,8.500000,36.555556,63.425001,...,10.305556,32.222222,60.444451,7.472222,4.305556,14.000000,36,14.0,2020.0,0.250000


In [150]:
for i in range(len(odds)):
    if odds['Home Win By (Vegas)'][i] == 'pk':
        odds['Home Win By (Vegas)'][i] = 0
        
    odds['Home Win By (Vegas)'][i] = float(odds['Home Win By (Vegas)'][i])

<ipython-input-150-c448c1f96edb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['Home Win By (Vegas)'][i] = float(odds['Home Win By (Vegas)'][i])
<ipython-input-150-c448c1f96edb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds['Home Win By (Vegas)'][i] = 0


In [151]:
all_stats['Home Win By (Vegas)'] = odds['Home Win By (Vegas)']

In [157]:
#make FT% columns
all_stats['Home FT%'] = all_stats['Home FTM']/all_stats['Home FTA']
all_stats['Away FT%'] = all_stats['Away FTM']/all_stats['Away FTA']

In [160]:
#drop FTM and FTA because we already made a FT% columns
all_stats.drop(columns=['Home FTM', 'Home FTA', 'Away FTM', 'Away FTA'], inplace=True)

In [161]:
#reorder columns to make entering stats easier
all_stats = all_stats[['Home GP', 'Home Win Rate', 'Home Points For', 'Home FT%', 'Home ORB', 'Home DRB', 'Home TOV', 
                       'Home STL', 'Home BLK', 'Home AST%', 'Home eFG%', 'Home FTr', 'Home 3PAr', 'Home Points Against',
                       'Away GP', 'Away Win Rate', 'Away Points For', 'Away FT%', 'Away ORB', 'Away DRB', 'Away TOV', 
                       'Away STL', 'Away BLK', 'Away AST%', 'Away eFG%', 'Away FTr', 'Away 3PAr', 'Away Points Against',
                       'Home Win By (Vegas)', 'Season', 'Home Win By']]

In [162]:
all_stats

,Home GP,Home Win Rate,Home Points For,Home FT%,Home ORB,Home DRB,Home TOV,Home STL,Home BLK,Home AST%,...,Away STL,Away BLK,Away AST%,Away eFG%,Away FTr,Away 3PAr,Away Points Against,Home Win By (Vegas),Season,Home Win By
2011-12-25 00:00:00 Boston @ New York,1,1.000000,106.000000,0.794118,8.000000,23.000000,16.000000,9.000000,11.000000,48.599998,...,7.000000,5.000000,71.800003,0.526000,0.408000,0.066000,106.000000,4.5,2011.0,2.0
2011-12-25 00:00:00 Chicago @ Lakers,1,0.000000,87.000000,0.550000,12.000000,30.000000,17.000000,6.000000,8.000000,61.099998,...,14.000000,4.000000,58.299999,0.444000,0.157000,0.169000,87.000000,-5.0,2011.0,-1.0
2011-12-25 00:00:00 Clippers @ Golden State,1,0.000000,86.000000,0.708333,17.000000,31.000000,16.000000,4.000000,8.000000,53.099998,...,9.000000,8.000000,59.500000,0.506000,0.469000,0.284000,86.000000,-4.5,2011.0,-19.0
2011-12-25 00:00:00 Miami @ Dallas,1,0.000000,94.000000,0.766667,8.000000,23.000000,17.000000,10.000000,0.000000,74.199997,...,9.000000,6.000000,47.400002,0.513000,0.462000,0.090000,94.000000,-4.5,2011.0,-11.0
2011-12-25 00:00:00 Orlando @ Oklahoma City,1,1.000000,97.000000,0.730769,7.000000,36.000000,16.000000,8.000000,5.000000,61.799999,...,7.000000,4.000000,60.000000,0.420000,0.321000,0.346000,97.000000,7.5,2011.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-18 00:00:00 Washington @ Boston,37,0.594595,114.189189,0.796935,10.783784,33.297297,14.135135,8.270270,5.081081,58.581088,...,7.405405,4.405405,60.143245,0.521459,0.284108,0.312378,117.864865,2.0,2020.0,18.0
2021-05-19 00:00:00 Golden State @ Lakers,37,0.594595,111.216216,0.745690,9.837838,35.351351,15.297297,8.108108,5.648649,61.097300,...,8.378378,4.945946,64.821612,0.534541,0.232541,0.441270,113.837838,5.5,2020.0,3.0
2021-05-19 00:00:00 San Antonio @ Memphis,37,0.513514,109.756757,0.774903,11.621622,35.729730,12.243243,8.783784,5.405405,63.818920,...,7.081081,5.081081,57.702703,0.511946,0.251000,0.307514,111.729730,4.5,2020.0,4.0
2021-05-20 00:00:00 Indiana @ Washington,37,0.540541,119.621622,0.763452,9.540541,36.189189,13.702703,7.189189,4.108108,57.748654,...,8.513514,6.324324,66.159457,0.552649,0.211919,0.376162,115.837838,3.5,2020.0,27.0


In [163]:
all_stats.to_csv('avg_by_game_2')